In [ ]:
import requests
import pandas as pd

base_url = "https://api.croma.com/searchservices/v1/search"

headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "en-US,en;q=0.9",
    "dnt": "1",
    "origin": "https://www.croma.com",
    "referer": "https://www.croma.com/",
    "sec-ch-ua": '"Not;A=Brand";v="99", "Google Chrome";v="139", "Chromium";v="139"',
    "sec-ch-ua-mobile": "?1",
    "sec-ch-ua-platform": '"Android"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/139.0.0.0 Mobile Safari/537.36"
}


all_products = []
page = 0
while True:
    params = {
        "currentPage": page,
        "query": "laptop:relevance",
        "fields": "FULL",
        "channel": "WEB",
        "channelCode": "370040",
        "spellOpt": "DEFAULT"
    }

    r = requests.get(base_url, headers=headers, params=params)
    
    if r.status_code != 200:
        print(f"Stopped at page {page}, status={r.status_code}")
        break
    
    data = r.json()
    products = data.get("products", [])
    
    if not products:
        print(f"No more products at page {page}")
        break

    print(f"Page {page}: {len(products)} products")
    
    for p in products:
        all_products.append({
            "Name": p.get("name"),
            "Brand": p.get("brandName"),
            "Price": p.get("price", {}).get("value"),
            "CPU": p.get("attributes", {}).get("Processor", None),
            "RAM": p.get("attributes", {}).get("RAM", None),
            "Storage": p.get("attributes", {}).get("Hard Disk Capacity", None),
            "GPU": p.get("attributes", {}).get("Graphics Processor", None),
            "URL": "https://www.croma.com" + p.get("url", "")
        })
    
    page += 1

df = pd.DataFrame(all_products)
df.to_csv("croma_laptops.csv", index=False)
print(f" Scraped {len(df)} products and saved to croma_laptops.csv")
